In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
from matplotlib.colors import ListedColormap


In [15]:
df = pd.read_csv('/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/Results/filtered_p99__rcp4p5.csv')

# Ensure that the 'lat' and 'long' columns are numeric
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')

# Drop rows with invalid coordinates
df.dropna(subset=['lat', 'long'], inplace=True)

# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Define the time periods
time_periods = ['historical', 'early', 'mid', 'end']
cmap = plt.get_cmap('tab10', 11)  # 'tab10' colormap has 10 distinct colors
colors = cmap.colors
cmap = ListedColormap(colors)


In [17]:
for period in time_periods:
    # Filter data for the current time period
    gdf_period = gdf[gdf['time_period'] == period]
    
    # Check if there's data for the current period
    if gdf_period.empty:
        print(f"No data available for {period.capitalize()}. Skipping...")
        continue
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_period.plot(ax=ax, 
                    column='cluster', 
                    cmap=cmap, 
                    legend=True, 
                    markersize=50,  # Adjust marker size as needed
                    alpha=0.7, 
                    edgecolor='k')
    

    
    # Set titles and labels
    ax.set_title(f'Map for {period.capitalize()}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Save the plot as a PNG file
    plt.savefig(f'rcp4p5_map_{period}.png', bbox_inches='tight')
    plt.close()
